In [1]:
import csv
from cv2 import imread,cvtColor
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
file = open('driver_imgs_list.csv')
lst = csv.reader(file)
file_list = []
for i in lst:
    file_list.append(i)
file_list.pop(0)
w = 128
h = 192
input_size = w * h

In [3]:
def read_file(file):
    img = cv2.imread('train/'+file[1]+'/'+file[2],0)
    img = cv2.resize(img,(w,h), interpolation=cv2.INTER_LINEAR)
    img = img.astype(np.float32)/255.0
    label = int(file[1][1])
    return img,label
def show(array):
    plt.figure()
    plt.imshow(array.reshape([h,w]), cmap="gray")
    plt.show()
def get_batch(batch_size):
    j = np.random.randint(0,20788,batch_size)
    batch = np.zeros((batch_size,h,w,1))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k,:,:,0],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label
def get_test_batch(batch_size):
    j = np.random.randint(20788,22424,batch_size)
    batch = np.zeros((batch_size,h,w,1))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k,:,:,0],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label

In [4]:
## 
X = tf.placeholder (dtype=tf.float32,shape=[None, h,w,1])
Y = tf.placeholder(dtype=tf.float32,shape=[None,10])
####
####
#con0= tf.layers.conv2d(inputs=X,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same')
#con1 = tf.layers.conv2d(inputs=con0,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same')
#max1 = tf.layers.max_pooling2d(inputs=con1,pool_size=2,strides=2)
####
con0 = tf.layers.conv2d(inputs=X,kernel_size=(7,7),filters=4,activation=tf.nn.relu,strides=1,padding='same')
max0 = tf.layers.max_pooling2d(inputs=con0,pool_size=2,strides=2)
########
con1 = tf.layers.conv2d(inputs=max0,kernel_size=(5,5),filters=8,activation=tf.nn.relu,strides=1,padding='same')
max1 = tf.layers.max_pooling2d(inputs=con1,pool_size=2,strides=2)
#####
con2 = tf.layers.conv2d(inputs=max1,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same')
max2 = tf.layers.max_pooling2d(inputs=con2,pool_size=2,strides=2)
#####
con3 = tf.layers.conv2d(inputs=max2,kernel_size=(3,3),filters=32,activation=tf.nn.relu,strides=1,padding='same')
max3 = tf.layers.max_pooling2d(inputs=con3,pool_size=2,strides=2)
#####
con4 = tf.layers.conv2d(inputs=max3,kernel_size=(3,3),filters=64,activation=tf.nn.relu,strides=1,padding='same')
max4 = tf.layers.max_pooling2d(inputs=con4,pool_size=2,strides=2)
#####
#######################################
fc_1 = tf.reshape(max4,[-1,6*64*4])
dr = tf.placeholder(dtype=tf.float32,shape=[])
fc_1_d = tf.nn.dropout(fc_1,keep_prob=dr)
#######################################
fc_2 = tf.layers.dense(inputs=fc_1_d,units=2000,activation=tf.nn.relu)
fc_2_d = tf.nn.dropout(fc_2,keep_prob=dr)
#######################################
fc_3 = tf.layers.dense(inputs=fc_2_d,units=10)
#######################################
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=fc_3))
optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
correctPred = tf.equal(tf.argmax(fc_3,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [6]:
for i in range(3000):
    batch_x,batch_y = get_batch(16)
    _,l,acc = sess.run([optimizer,loss,accuracy],feed_dict={X: batch_x,Y:batch_y,dr:0.5})
    if i % 100 == 0:
        print(l,acc)
        total_acc = 0
        for i in range(100):
            batch_x,batch_y = get_test_batch(16)
            l,acc = sess.run([loss,accuracy],feed_dict={X: batch_x,Y:batch_y,dr:1})
            total_acc += acc
        print(total_acc/100)

2.3063211 0.0625
0.1075
2.3705945 0.0625
0.100625
2.2546272 0.125
0.093125
2.3030689 0.0
0.179375
2.0238442 0.125
0.2425
1.8043628 0.375
0.313125
1.4000931 0.5625
0.283125
1.5484152 0.5625
0.274375
1.5369716 0.5
0.22875
0.94950867 0.75
0.31375
1.0569872 0.6875
0.246875
0.6551057 0.9375
0.251875
1.1126161 0.6875
0.25625
1.171813 0.625
0.278125
1.0633109 0.4375
0.32375
0.9989351 0.5625
0.261875
0.60353553 0.8125
0.266875
0.6252892 0.75
0.25875
0.2317172 0.9375
0.270625
0.3985921 0.875
0.31875
0.31424722 0.9375
0.281875
0.90661037 0.8125
0.279375
0.8526575 0.8125
0.3425
0.36889836 0.875
0.335625
0.43739894 0.875
0.320625
0.7549963 0.625
0.3375
1.024776 0.75
0.3325
0.6157752 0.75
0.32375
0.50248957 0.875
0.305
0.32265174 0.875
0.29


In [7]:
total_acc = 0
for i in range(100):
    batch_x,batch_y = get_test_batch(16)
    l,acc = sess.run([loss,accuracy],feed_dict={X: batch_x,Y:batch_y,dr:1})
    total_acc += acc
print(total_acc/100)

0.336875
